In [1]:
from gensim.corpora import Dictionary, WikiCorpus, MmCorpus

In [2]:
import sys
import time
sys.path.append('/Users/v.khodyreva/anaconda3/envs/py36/lib/python3.6/site-packages/gensim/corpora/')
import itertools

from wikicorpus import *

In [3]:
def extract_pages(f, filter_namespaces=False, filter_articles=None):
    """Extract pages from a MediaWiki database dump.

    Parameters
    ----------
    f : file
        File-like object.
    filter_namespaces : list of str or bool
         Namespaces that will be extracted.

    Yields
    ------
    tuple of (str or None, str, str)
        Title, text and page id.

    """
    elems = (elem for _, elem in iterparse(f, events=("end",)))

    # We can't rely on the namespace for database dumps, since it's changed
    # it every time a small modification to the format is made. So, determine
    # those from the first element we find, which will be part of the metadata,
    # and construct element paths.
    elem = next(elems)
    namespace = get_namespace(elem.tag)
    ns_mapping = {"ns": namespace}
    page_tag = "{%(ns)s}page" % ns_mapping
    text_path = "./{%(ns)s}revision/{%(ns)s}text" % ns_mapping
    title_path = "./{%(ns)s}title" % ns_mapping
    ns_path = "./{%(ns)s}ns" % ns_mapping
    pageid_path = "./{%(ns)s}id" % ns_mapping
    parentid_path = "./{%(ns)s}revision/{%(ns)s}parentid" % ns_mapping
    timestamp_path = "./{%(ns)s}revision/{%(ns)s}timestamp" % ns_mapping
    
    for elem in elems:
        if elem.tag == page_tag:
            title = elem.find(title_path).text
            text = elem.find(text_path).text
            parentid_res = elem.find(parentid_path)
            try:
                parentid = parentid_res.text
            except:
                parentid = ''
            timestamp_res = elem.find(timestamp_path)
            try:
                timestamp = timestamp_res.text
            except:
                timestamp = ''

            if filter_namespaces:
                ns = elem.find(ns_path).text
                if ns not in filter_namespaces:
                    text = None

            if filter_articles is not None:
                if not filter_articles(
                        elem, namespace=namespace, title=title,
                        text=text, page_tag=page_tag,
                        text_path=text_path, title_path=title_path,
                        ns_path=ns_path, pageid_path=pageid_path):
                    text = None

            pageid = elem.find(pageid_path).text
            yield title, text or "", pageid, parentid, timestamp  # empty page will yield None

            # Prune the element tree, as per
            # http://www.ibm.com/developerworks/xml/library/x-hiperfparse/
            # except that we don't need to prune backlinks from the parent
            # because we don't use LXML.
            # We do this only for <page>s, since we need to inspect the
            # ./revision/text element. The pages comprise the bulk of the
            # file, so in practice we prune away enough.
            elem.clear()

def tokenize(content):
    # override original method in wikicorpus.py
    return [token for token in utils.tokenize(content, lower=True, errors='ignore')
            if len(token) <= 15 and not token.startswith('_')]

def process_article(args):
   # override original method in wikicorpus.py
    text, lemmatize, title, pageid, parentid, timestamp = args
    text = filter_wiki(text)
    result = text
#     result = pipeline.fit_transform([text])[0]
#     if lemmatize:
#         result = utils.lemmatize(text)
#     else:
#         result = tokenize(text)
    return result, title, pageid, parentid, timestamp


class MyWikiCorpus(WikiCorpus):
    def __init__(self, fname, processes=None, lemmatize=utils.has_pattern(), dictionary=None, 
                 filter_namespaces=('0',), tokenizer_func=tokenize, article_min_tokens=ARTICLE_MIN_WORDS):
        WikiCorpus.__init__(self, fname, processes, lemmatize, dictionary, filter_namespaces, 
                            tokenizer_func, article_min_tokens)
        self.metadata = True

    def get_texts(self):
        articles, articles_all = 0, 0
        positions, positions_all = 0, 0
        texts = ((text, self.lemmatize, title, pageid, parentid, timestamp) 
                 for title, text, pageid, parentid, timestamp in 
                 extract_pages(bz2.BZ2File(self.fname), self.filter_namespaces))
        pool = multiprocessing.Pool(self.processes)
        # process the corpus in smaller chunks of docs, because multiprocessing.Pool
        # is dumb and would load the entire input into RAM at once...
        for group in utils.chunkize(texts, chunksize=10 * self.processes, maxsize=1):
            for tokens, title, pageid, parentid, timestamp in pool.imap(process_article, group):  # chunksize=10):
                articles_all += 1
                positions_all += len(tokens)
                # article redirects and short stubs are pruned here
                if len(tokens) < ARTICLE_MIN_WORDS or any(title.startswith(ignore + ':') for ignore in IGNORED_NAMESPACES):
                    continue
                articles += 1
                positions += len(tokens)
                if self.metadata:
                    yield (tokens, (pageid, title, parentid, timestamp))
                else:
                    yield tokens
        pool.terminate()

        logger.info(
            "finished iterating over Wikipedia corpus of %i documents with %i positions"
            " (total %i articles, %i positions before pruning articles shorter than %i words)",
            articles, positions, articles_all, positions_all, ARTICLE_MIN_WORDS)
        self.length = articles  # cache corpus length

In [4]:
import sys
import warnings
import pandas as pd

sys.path.append('../src')
warnings.simplefilter('ignore')

from preprocessing import *
from sklearn.pipeline import Pipeline as sklearn_pipeline

chars_map = {'\xad': ' ',
             '\n': ' ',
             '…': '...',
             '«': '', '»': '',
             '"': '', '\'': '',
             '’': '', '‘': '',
             '”': '', '“': '', '„': '',
             '`': '', '*': '', '_': '',
             'http://':'', 'https://':''}
upos_set={'ADJ', 'ADV', 'INTJ', 'NOUN', 'PROPN', 'VERB'}

pipeline = sklearn_pipeline([('replace_chars', ReplaceChars(chars_map)),
                             ('sub_code', RegExprSub(r'\{.*\}', ' ')),
                             ('sub_colon', RegExprSub(r'\d*\:\d*', ' ')),
                             ('sub_spaces', RegExprSub(r' +', ' ')),
                             ('strip', Strip()),
                             ('sent_tokenize', RusSentTokenizer()),
#                              ('spell', Yandex_Speller()),
                             ('word_tokenize', Spacy_RusWordTokenizer()),
                             ('space_detect', SpaceDetecter()),
                             ('morph_predict', MorphPredictor()),
                             ('morph_filtration', MorphFilter(upos_set=upos_set)),
                             ('vowpal_wabbit_encode', VowpalWabbitFormatEncoder())])
#                              ('conllu_encode', CoNLLUFormatEncoder()),
#                              ('syntax_parse', SyntaxParser('../models/parser_model.udpipe'))])

Using TensorFlow backend.


In [5]:
dump_file = '../../../Desktop/Wiki/wikidump/ruwiki-20190820-pages-articles-multistream.xml.bz2'

dictionary = Dictionary()
wiki = MyWikiCorpus(dump_file, dictionary=dictionary)
wiki_texts = wiki.get_texts()

In [ ]:
http://www.cs.technion.ac.il/~gabr/resources/code/wikiprep/
https://kodingnotes.wordpress.com/2014/12/03/parsing-wikipedia-page-hierarchy/

In [ ]:
start = time.time()
res = []
with open('../../../Desktop/Wiki/wiki_log_3.txt', 'a') as f:
    for i, t in enumerate(wiki_texts):
        if i<842400:
            continue
        wv_line = pipeline.fit_transform([t[0]])[0]
        f.write(wv_line + '|||' + '|||'.join(t[1]) + '\n')
        if i % 100 == 0:
            print(i, t[1])
end = time.time()

842400 ('3622723', 'Число Воббе', '89483853', '2019-07-02T00:29:48Z')
842500 ('3623917', 'БГПИ — БГПА — БНТУ', '', '2011-12-18T14:29:22Z')
842600 ('3624846', 'Бивер (Фолклендские острова)', '76205512', '2017-09-20T05:47:23Z')
842700 ('3625771', 'Анна Каренина (МХАТ)', '101612144', '2019-08-15T04:46:45Z')
842800 ('3626267', 'Валюта Евразийского экономического союза', '98087312', '2019-02-13T13:21:44Z')
842900 ('3627080', 'Братья Чудновские', '98662448', '2019-07-21T15:10:23Z')
843000 ('3627621', 'Хронологический список католических блаженных и святых XVII века', '93138197', '2019-01-16T04:30:59Z')
843100 ('3628015', '50-й отдельный дивизион бронепоездов', '93833647', '2018-07-09T11:23:53Z')
843200 ('3628448', 'Польская академия литературы', '76572908', '2016-07-09T18:22:17Z')
843300 ('3628876', 'Юные чапаевцы (отряд)', '89052705', '2018-05-04T10:24:14Z')
843400 ('3629407', 'Изумрудный квартал', '98815032', '2019-03-24T10:10:12Z')
843500 ('3629923', 'PocketStudio', '92669202', '2019-07-0

852800 ('3670498', 'Международный теннисный турнир в Аделаиде 2008', '93011215', '2019-03-17T17:59:28Z')
852900 ('3670968', 'Водяной скорпион обыкновенный', '56071629', '2014-08-10T01:15:20Z')
853000 ('3671396', 'Полевая улица (Самара)', '85729098', '2019-05-28T21:02:07Z')
853100 ('3671753', 'Сандерс, Георгий', '89416484', '2019-06-05T10:35:03Z')
853200 ('3672269', 'Логёйа', '98070478', '2019-05-10T09:15:05Z')
853300 ('3672649', 'Гаврилин, Николай Митрофанович', '94926468', '2019-04-16T21:54:15Z')
853400 ('3673149', 'Моргхан', '58165408', '2017-09-30T11:45:12Z')
853500 ('3673676', 'Образование в Сербии', '97490578', '2019-07-26T17:48:38Z')
853600 ('3674210', 'Федотовская дорожка', '79655386', '2017-09-10T22:10:17Z')
853700 ('3674728', '133 км (платформа Рижского направления)', '85277865', '2019-08-13T18:39:14Z')
853800 ('3675268', 'Ню-Фрисланд', '61146722', '2016-04-16T20:33:56Z')
853900 ('3675681', 'Болотница (растение)', '90605717', '2018-02-12T21:33:02Z')
854000 ('3676165', 'Ички (Р

863000 ('3721141', 'После нашей эры', '98693935', '2019-05-29T15:10:25Z')
863100 ('3721556', 'Ренар, Эрве', '101322232', '2019-07-29T21:24:04Z')
863200 ('3722064', 'Мама Люба', '101435290', '2019-08-11T17:51:23Z')
863300 ('3722465', 'Степанов, Георгий Юрьевич', '100431305', '2019-06-15T21:40:12Z')
863400 ('3722908', 'Кларамунт, Хосе', '93788918', '2018-07-06T16:17:55Z')
863500 ('3723374', 'Бирмингемские беспорядки', '87751425', '2019-04-25T12:46:45Z')
863600 ('3723858', 'Во власти золота', '93085259', '2018-09-14T01:04:00Z')
863700 ('3724332', 'Вестландский буревестник', '68580547', '2017-01-01T23:40:52Z')
863800 ('3724798', 'Голям-Извор', '68443107', '2016-02-23T08:07:44Z')
863900 ('3725264', 'Патентное право Украины', '100044601', '2019-06-10T11:44:18Z')
864000 ('3725658', 'Российский союз ректоров', '94061406', '2019-02-04T08:23:01Z')
864100 ('3726110', 'Баласма', '92891666', '2018-06-10T19:04:22Z')
864200 ('3726595', 'Керан (Чехармехаль и Бахтиария)', '58166148', '2017-09-30T02:45:

873400 ('3766377', 'Кршивицкий, Николай Фаддеевич', '81258361', '2017-01-25T17:45:21Z')
873500 ('3766778', 'Очаровательный листолаз', '89427540', '2018-11-02T00:26:36Z')
873600 ('3767206', 'Тромбофилия', '100737152', '2019-07-07T14:44:30Z')
873700 ('3767582', 'Музей обороны Туапсе', '100446349', '2019-08-15T18:07:54Z')
873800 ('3768021', 'Жундия', '53955606', '2016-02-23T08:52:37Z')
873900 ('3768379', 'Памятник Льву Ошанину', '95005447', '2018-11-02T13:27:46Z')
874000 ('3768680', 'Марисоль (телесериал)', '93965629', '2018-07-24T23:04:52Z')
874100 ('3769066', 'Джилкс, Мэттью', '101201464', '2019-07-23T10:39:20Z')
874200 ('3769518', 'Морозовский', '68909282', '2016-02-23T11:29:04Z')
874300 ('3770018', 'Гарни, Уолтер', '83019843', '2017-09-30T03:37:11Z')
874400 ('3770463', 'Спасо-Преображенский Новгород-Северский мужской монастырь', '42605957', '2012-07-26T06:31:20Z')
874500 ('3770948', 'Mark 54 (торпеда)', '81927587', '2017-09-08T23:42:01Z')
874600 ('3771264', 'Бумеранг (колёсная платфор

884100 ('3816230', 'Мария Антония Испанская', '99372180', '2019-06-02T10:23:15Z')
884200 ('3816803', 'Национальный музей Индии', '91344705', '2019-06-16T07:22:44Z')
884300 ('3817287', 'Первая лига Латвии по футболу 2007', '101175590', '2019-08-13T20:07:18Z')
884400 ('3817764', 'Square Kilometre Array', '99356886', '2019-07-13T07:47:28Z')
884500 ('3818097', 'Польский Вудсток', '101376847', '2019-08-01T19:35:43Z')
884600 ('3818423', 'Borderlands 2', '101409929', '2019-08-09T17:02:44Z')
884700 ('3818907', 'Лолойские языки', '87038992', '2018-06-08T12:31:01Z')
884800 ('3819391', 'Бороздин, Андрей Михайлович', '99062051', '2019-04-25T12:43:52Z')
884900 ('3819988', 'Палмотич, Джунье', '79853244', '2019-02-24T16:36:23Z')
885000 ('3820342', 'Цриевич Туберон, Людовик', '63374840', '2015-12-23T21:45:06Z')
885100 ('3820816', 'Сельское поселение Подгорное (Борский район Самарской области)', '47987191', '2012-09-08T22:03:54Z')
885200 ('3821297', '85-я инженерная бригада', '81600171', '2019-04-13T00

...

1008800 ('4332265', 'Citi Open 2012 — женский одиночный разряд', '67523403', '2015-03-21T21:50:53Z')
1008900 ('4332558', 'Наумовская (Верховское сельское поселение)', '69461436', '2018-10-24T01:58:48Z')
1009000 ('4332680', 'Милогорская', '86421893', '2018-07-28T16:55:37Z')
1009100 ('4332801', 'Малаховский Бор', '94214582', '2019-05-08T07:42:24Z')
1009200 ('4333178', 'Орловский уезд (древний)', '', '2013-01-16T13:40:19Z')
1009300 ('4333533', 'Атенино', '87242896', '2018-01-02T08:12:41Z')
1009400 ('4333820', 'Японская зарянка', '79282569', '2017-10-27T03:38:23Z')
1009500 ('4334261', 'Toutes les femmes en moi', '97272921', '2019-02-03T09:00:09Z')
1009600 ('4334576', 'Узбекистан на летних Паралимпийских играх 2012', '91552586', '2018-06-27T06:55:49Z')
1009700 ('4335062', 'Пиреноид', '97489130', '2019-04-13T23:53:48Z')
1009800 ('4335376', 'МИД Грузии', '', '2013-01-17T21:28:11Z')
1009900 ('4335691', 'Голдрайх', '51738395', '2013-03-18T03:59:10Z')
1010000 ('4336084', 'Любятово (значения)', '

1019100 ('4368700', 'Ростов-на-Дону (значения)', '74997170', '2019-04-03T17:02:57Z')
1019200 ('4368956', 'Немоловский', '71617664', '2019-01-15T18:19:01Z')
1019300 ('4369370', 'Наментенга', '89409438', '2018-10-23T11:22:42Z')
1019400 ('4369693', 'Image-Line Software', '101453838', '2019-08-06T05:44:24Z')
1019500 ('4369949', 'Список умерших в 1124 году', '73772185', '2017-02-28T14:41:31Z')
1019600 ('4370160', 'Покахонтас 2: Путешествие в Новый Свет', '95329833', '2018-12-23T17:50:31Z')
1019700 ('4370475', 'Список депутатов Верховного Совета БАССР 3-го созыва', '92176421', '2018-11-23T02:14:31Z')
1019800 ('4370896', 'Противопожарные разрывы', '90735556', '2018-02-27T13:47:00Z')
1019900 ('4371212', 'Террористический акт на острове Утойя', '69031406', '2015-12-18T23:00:13Z')
1020000 ('4371560', 'Епархия Минделу', '67732081', '2016-08-04T05:16:28Z')
1020100 ('4372059', 'Гетто в Осиповичском районе', '52358053', '2013-06-19T11:27:03Z')
1020200 ('4372402', 'Гришин, Виктор Иванович (ректор)', 

1029300 ('4404427', 'Штайнмец, Миклош', '92812172', '2019-02-15T13:24:08Z')
1029400 ('4404739', 'Гурзаул', '88570288', '2018-02-12T20:54:09Z')
1029500 ('4405017', 'Братская улица (Екатеринбург)', '85435943', '2019-07-20T23:44:45Z')
1029600 ('4405316', 'Каратал (Ерейментауский район)', '87620070', '2017-10-18T05:03:58Z')
1029700 ('4405585', 'Stigmella satarensis', '75949543', '2018-05-17T01:53:11Z')
1029800 ('4406039', 'Гандольфи, Хавьер', '96739368', '2019-01-16T00:38:35Z')
1029900 ('4406380', 'Гарфилд, Ричард', '99566776', '2019-05-03T12:38:01Z')
1030000 ('4406694', 'Бразилия за Христа', '', '2013-03-01T17:50:07Z')
1030100 ('4406932', 'Список глав государств в 402 году', '82209456', '2017-05-19T15:56:36Z')
1030200 ('4407248', 'Евсеев, Николай Иванович', '98047623', '2019-07-20T12:11:22Z')
1030300 ('4407442', 'HD 197037', '52940537', '2013-03-03T20:30:16Z')
1030400 ('4407774', 'Айгерим Жексембинова', '52929838', '2014-11-20T16:54:04Z')
1030500 ('4407988', 'Байтерек (Акмолинская область

1039800 ('4443444', 'Хаслох (значения)', '54187260', '2016-02-22T20:44:14Z')
1039900 ('4443707', 'Краденое солнце (мультфильм)', '100046648', '2019-07-06T02:48:18Z')
1040000 ('4443890', 'Alfa Corse', '87426977', '2018-01-09T06:03:59Z')
1040100 ('4444267', 'Сокуров, Сергей Анатольевич', '100329745', '2019-07-26T17:26:36Z')
1040200 ('4444679', 'Хейнясенмаа (значения)', '', '2013-03-25T12:24:55Z')
1040300 ('4444953', 'Ганахлеба (Гальский район)', '86131606', '2017-08-21T11:13:53Z')
1040400 ('4445406', 'Dynasty Warriors 8', '93882843', '2019-06-22T13:16:36Z')
1040500 ('4445840', 'Ясотхон (провинция)', '62920175', '2018-07-23T08:15:02Z')
1040600 ('4446109', 'Прачуапкхирикхан (значения)', '54231946', '2013-04-04T18:37:13Z')
1040700 ('4446403', 'Блошница', '84414520', '2018-01-30T16:59:06Z')
1040800 ('4446722', 'Де Кадене, Аманда', '97051283', '2019-03-16T08:11:19Z')
1040900 ('4447109', 'Подхоз ПЧ-45', '66705168', '2017-11-24T02:00:20Z')
1041000 ('4447474', 'Власов, Трофим Леонтьевич', '91121

1050100 ('4481623', 'Лачинов, Михаил Фёдорович', '58141146', '2019-01-12T22:32:20Z')
1050200 ('4482155', 'Осада Мангупа', '86145499', '2017-09-10T21:00:43Z')
1050300 ('4482517', 'Удмурт-Сада', '89437978', '2019-06-22T00:32:03Z')
1050400 ('4483132', 'The Rubens', '93691264', '2018-09-01T01:15:00Z')
1050500 ('4483598', 'Шпаль, Милан', '80646608', '2016-12-03T16:54:41Z')
1050600 ('4484057', 'Ордена Приднестровской Молдавской Республики', '', '2013-04-19T16:07:21Z')
1050700 ('4484416', 'Dinoponera snellingi', '68545254', '2018-07-09T16:50:21Z')
1050800 ('4484810', 'Клыковская', '91461728', '2018-07-07T03:31:18Z')
1050900 ('4485249', 'Калливере', '100519095', '2019-06-20T19:30:23Z')
1051000 ('4485663', 'Пеи-де-Бюш', '86245918', '2019-05-09T08:20:07Z')
1051100 ('4486109', 'Масонский запон', '96549926', '2019-07-17T07:53:54Z')
1051200 ('4486527', 'HD 120084', '55780203', '2016-07-12T19:32:22Z')
1051300 ('4486888', 'Эзна (значения)', '', '2013-04-21T16:47:49Z')
1051400 ('4487229', 'Улица Мёдов

In [9]:
data = []
with open('../../../Desktop/Wiki/wiki_log.txt', 'r') as f:
    for i, line in enumerate(f):
        parts = line.strip().split('|||')
        data.append(parts)
        if i % 1000 == 0:
            print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
